In [1]:
import lightgbm
import pickle
import pandas as pd
import numpy as np
import gc

In [2]:
# training set
x = pd.read_csv("data/dummy/train.csv")

In [3]:
# validation set
x_val = x.loc[x['srch_id']%10 == 0]
x = x.drop(x_val.index)

In [4]:
# drop columns which cannot be used for both train and val
x.drop(['click_bool', 'booking_bool', 'date_time', 'random_bool'], axis=1, inplace=True)
x_val.drop(['click_bool', 'booking_bool', 'date_time', 'random_bool'], axis=1, inplace=True)

# sort both on search id
x.sort_values('srch_id', ignore_index=True)
x_val.sort_values('srch_id', ignore_index=True)

,srch_id,site_id,visitor_location_country_id,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,...,prop_review_score_srch_id_std,prop_location_score1_srch_id_std,prop_location_score2_srch_id_mean,prop_location_score2_srch_id_std,prop_location_score2_srch_id_median,price_usd_srch_destination_id_median,prop_location_score2_srch_destination_id_mean,prop_location_score2_srch_destination_id_std,prop_location_score2_srch_destination_id_median,target
0,30,24,216,219,13155,3,3.5,1,4.76,0.3273,...,0.886965,0.809980,0.300107,0.153532,0.3588,198.67,0.325985,0.133087,0.366,0
1,30,24,216,219,137592,3,4.0,1,4.61,0.2374,...,0.886965,0.809980,0.300107,0.153532,0.3588,198.67,0.325985,0.133087,0.366,0
2,30,24,216,219,136754,3,0.0,0,4.81,0.0000,...,0.886965,0.809980,0.300107,0.153532,0.3588,198.67,0.325985,0.133087,0.366,0
3,30,24,216,219,133729,3,3.5,1,4.77,0.3932,...,0.886965,0.809980,0.300107,0.153532,0.3588,198.67,0.325985,0.133087,0.366,0
4,30,24,216,219,130968,4,4.5,1,4.81,0.3536,...,0.886965,0.809980,0.300107,0.153532,0.3588,198.67,0.325985,0.133087,0.366,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
503647,332750,5,219,219,58095,2,3.0,0,1.39,0.0004,...,0.636634,1.092423,0.039761,0.043743,0.0189,140.00,0.044633,0.039330,0.035,0
503648,332750,5,219,219,58358,3,4.5,1,2.94,0.0074,...,0.636634,1.092423,0.039761,0.043743,0.0189,140.00,0.044633,0.039330,0.035,0
503649,332750,5,219,219,64324,2,4.0,1,0.00,0.0003,...,0.636634,1.092423,0.039761,0.043743,0.0189,140.00,0.044633,0.039330,0.035,0
503650,332750,5,219,219,40869,4,4.0,1,2.08,0.0125,...,0.636634,1.092423,0.039761,0.043743,0.0189,140.00,0.044633,0.039330,0.035,0


In [5]:
# make targets
y_val = x_val.target.squeeze()
y = x.target.squeeze()

# remove target column from train and val
x.drop(['target'], axis=1, inplace=True)
x_val.drop(['target'], axis=1, inplace=True)

In [6]:
groups = x.srch_id.value_counts().sort_index()
groups_val = x_val.srch_id.value_counts().sort_index()

In [7]:
# remove search id
x.drop(['srch_id'], axis=1, inplace=True)
x_val.drop(['srch_id'], axis=1, inplace=True)

In [8]:
pd.set_option('display.max_columns', None)
x.head(n=1)

,site_id,visitor_location_country_id,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,price_usd,promotion_flag,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,orig_destination_distance,agg_comp_rate,agg_comp_perc,agg_comp_inv,total,price_usd_prop_id_mean,price_usd_prop_id_std,price_usd_prop_id_median,prop_starrating_prop_id_mean,prop_review_score_prop_id_mean,prop_location_score1_prop_id_mean,prop_location_score1_prop_id_median,prop_location_score2_prop_id_mean,prop_location_score2_prop_id_std,prop_location_score2_prop_id_median,price_usd_srch_id_mean,price_usd_srch_id_std,price_usd_srch_id_median,prop_starrating_srch_id_std,prop_review_score_srch_id_std,prop_location_score1_srch_id_std,prop_location_score2_srch_id_mean,prop_location_score2_srch_id_std,prop_location_score2_srch_id_median,price_usd_srch_destination_id_median,prop_location_score2_srch_destination_id_mean,prop_location_score2_srch_destination_id_std,prop_location_score2_srch_destination_id_median
0,12,187,219,893,3,3.5,1,2.83,0.0438,4.95,104.77,0,23246,1,0,4,0,1,1,-1.0,0.0,0.0,0.0,4,118.758742,17.778734,118.0,3,3.5,2.83,2.83,0.053879,0.031432,0.0438,163.718929,96.355255,139.05,0.766356,1.109572,0.518734,0.045493,0.047216,0.02255,118.46,0.037571,0.043714,0.0155


In [9]:
categorical_columns = [0,1,2,3,6,11,12,18]#[1,2,3,4,7,12,13,19]
x.iloc[:,categorical_columns]

,site_id,visitor_location_country_id,prop_country_id,prop_id,prop_brand_bool,promotion_flag,srch_destination_id,srch_saturday_night_bool
0,12,187,219,893,1,0,23246,1
1,12,187,219,10404,1,0,23246,1
2,12,187,219,21315,1,0,23246,1
3,12,187,219,27348,1,0,23246,1
4,12,187,219,29604,1,0,23246,1
...,...,...,...,...,...,...,...,...
4958342,5,219,219,77700,1,0,16974,0
4958343,5,219,219,88083,1,0,16974,0
4958344,5,219,219,94508,1,0,16974,0
4958345,5,219,219,128360,1,0,16974,0


In [10]:
# default lightgbm model with sklearn api
gbm = lightgbm.LGBMRanker(metric="ndcg", objective="lambdarank", num_boost_round=3000, extra_trees=True,
                         seed=12345, deterministic=True)#, #learning_rate = 0.05)#, num_leaves=50) 

## nieuw: extra_trees
# fitting model 
gbm.fit(
    x,
    y,
    eval_set = [(x_val, y_val)],
    group = groups,
    eval_group = [groups_val],
    verbose=1,
    early_stopping_rounds=500,
    categorical_feature=categorical_columns
)

/home/susy/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:151: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/susy/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/susy/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1554: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [0, 1, 2, 3, 6, 11, 12, 18]
  warnings.warn('categorical_feature in Dataset is overridden.\n'


[LightGBM] [Warning] num_iterations is set=3000, num_boost_round=3000 will be ignored. Current value: num_iterations=3000


/home/susy/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/susy/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


[1]	valid_0's ndcg@1: 0.114114	valid_0's ndcg@2: 0.17257	valid_0's ndcg@3: 0.211998	valid_0's ndcg@4: 0.243041	valid_0's ndcg@5: 0.267712
Training until validation scores don't improve for 500 rounds
[2]	valid_0's ndcg@1: 0.141328	valid_0's ndcg@2: 0.205013	valid_0's ndcg@3: 0.245519	valid_0's ndcg@4: 0.276911	valid_0's ndcg@5: 0.301371
[3]	valid_0's ndcg@1: 0.15794	valid_0's ndcg@2: 0.226813	valid_0's ndcg@3: 0.268523	valid_0's ndcg@4: 0.299662	valid_0's ndcg@5: 0.322989
[4]	valid_0's ndcg@1: 0.163108	valid_0's ndcg@2: 0.231868	valid_0's ndcg@3: 0.274281	valid_0's ndcg@4: 0.304813	valid_0's ndcg@5: 0.329313
[5]	valid_0's ndcg@1: 0.169013	valid_0's ndcg@2: 0.238936	valid_0's ndcg@3: 0.281401	valid_0's ndcg@4: 0.312934	valid_0's ndcg@5: 0.3376
[6]	valid_0's ndcg@1: 0.169755	valid_0's ndcg@2: 0.241685	valid_0's ndcg@3: 0.283402	valid_0's ndcg@4: 0.315574	valid_0's ndcg@5: 0.339963
[7]	valid_0's ndcg@1: 0.172886	valid_0's ndcg@2: 0.24406	valid_0's ndcg@3: 0.286882	valid_0's ndcg@4: 0.3189

[60]	valid_0's ndcg@1: 0.200899	valid_0's ndcg@2: 0.277713	valid_0's ndcg@3: 0.324155	valid_0's ndcg@4: 0.355871	valid_0's ndcg@5: 0.380317
[61]	valid_0's ndcg@1: 0.20029	valid_0's ndcg@2: 0.277475	valid_0's ndcg@3: 0.323879	valid_0's ndcg@4: 0.356112	valid_0's ndcg@5: 0.380034
[62]	valid_0's ndcg@1: 0.200197	valid_0's ndcg@2: 0.277934	valid_0's ndcg@3: 0.32464	valid_0's ndcg@4: 0.356793	valid_0's ndcg@5: 0.380691
[63]	valid_0's ndcg@1: 0.200693	valid_0's ndcg@2: 0.27817	valid_0's ndcg@3: 0.325046	valid_0's ndcg@4: 0.357316	valid_0's ndcg@5: 0.381097
[64]	valid_0's ndcg@1: 0.200892	valid_0's ndcg@2: 0.278792	valid_0's ndcg@3: 0.325102	valid_0's ndcg@4: 0.357447	valid_0's ndcg@5: 0.381218
[65]	valid_0's ndcg@1: 0.201036	valid_0's ndcg@2: 0.279429	valid_0's ndcg@3: 0.32556	valid_0's ndcg@4: 0.357618	valid_0's ndcg@5: 0.381194
[66]	valid_0's ndcg@1: 0.201092	valid_0's ndcg@2: 0.278979	valid_0's ndcg@3: 0.325481	valid_0's ndcg@4: 0.357491	valid_0's ndcg@5: 0.381244
[67]	valid_0's ndcg@1: 0

...

[1294]	valid_0's ndcg@1: 0.211499	valid_0's ndcg@2: 0.293491	valid_0's ndcg@3: 0.340954	valid_0's ndcg@4: 0.373288	valid_0's ndcg@5: 0.397725
[1295]	valid_0's ndcg@1: 0.211449	valid_0's ndcg@2: 0.293472	valid_0's ndcg@3: 0.340886	valid_0's ndcg@4: 0.373241	valid_0's ndcg@5: 0.39764
[1296]	valid_0's ndcg@1: 0.2115	valid_0's ndcg@2: 0.29346	valid_0's ndcg@3: 0.340885	valid_0's ndcg@4: 0.373308	valid_0's ndcg@5: 0.39772
[1297]	valid_0's ndcg@1: 0.211451	valid_0's ndcg@2: 0.293473	valid_0's ndcg@3: 0.340898	valid_0's ndcg@4: 0.373342	valid_0's ndcg@5: 0.3976
[1298]	valid_0's ndcg@1: 0.211401	valid_0's ndcg@2: 0.293392	valid_0's ndcg@3: 0.340867	valid_0's ndcg@4: 0.373311	valid_0's ndcg@5: 0.39755
[1299]	valid_0's ndcg@1: 0.211401	valid_0's ndcg@2: 0.293392	valid_0's ndcg@3: 0.340842	valid_0's ndcg@4: 0.373307	valid_0's ndcg@5: 0.397527
[1300]	valid_0's ndcg@1: 0.2115	valid_0's ndcg@2: 0.29339	valid_0's ndcg@3: 0.340889	valid_0's ndcg@4: 0.373334	valid_0's ndcg@5: 0.397535
[1301]	valid_0's 

[1352]	valid_0's ndcg@1: 0.2113	valid_0's ndcg@2: 0.293482	valid_0's ndcg@3: 0.341264	valid_0's ndcg@4: 0.373532	valid_0's ndcg@5: 0.397537
[1353]	valid_0's ndcg@1: 0.211101	valid_0's ndcg@2: 0.293446	valid_0's ndcg@3: 0.341179	valid_0's ndcg@4: 0.373459	valid_0's ndcg@5: 0.39751
[1354]	valid_0's ndcg@1: 0.211101	valid_0's ndcg@2: 0.293477	valid_0's ndcg@3: 0.341185	valid_0's ndcg@4: 0.373508	valid_0's ndcg@5: 0.397559
[1355]	valid_0's ndcg@1: 0.211101	valid_0's ndcg@2: 0.293364	valid_0's ndcg@3: 0.341077	valid_0's ndcg@4: 0.373508	valid_0's ndcg@5: 0.397517
[1356]	valid_0's ndcg@1: 0.21115	valid_0's ndcg@2: 0.29335	valid_0's ndcg@3: 0.341138	valid_0's ndcg@4: 0.373547	valid_0's ndcg@5: 0.397557
[1357]	valid_0's ndcg@1: 0.210952	valid_0's ndcg@2: 0.293265	valid_0's ndcg@3: 0.341197	valid_0's ndcg@4: 0.373498	valid_0's ndcg@5: 0.397575
[1358]	valid_0's ndcg@1: 0.210903	valid_0's ndcg@2: 0.293247	valid_0's ndcg@3: 0.341228	valid_0's ndcg@4: 0.373444	valid_0's ndcg@5: 0.397591
[1359]	vali

[1410]	valid_0's ndcg@1: 0.211253	valid_0's ndcg@2: 0.293207	valid_0's ndcg@3: 0.340747	valid_0's ndcg@4: 0.373097	valid_0's ndcg@5: 0.397438
[1411]	valid_0's ndcg@1: 0.211451	valid_0's ndcg@2: 0.293542	valid_0's ndcg@3: 0.340858	valid_0's ndcg@4: 0.373201	valid_0's ndcg@5: 0.397562
[1412]	valid_0's ndcg@1: 0.211451	valid_0's ndcg@2: 0.293542	valid_0's ndcg@3: 0.340882	valid_0's ndcg@4: 0.373183	valid_0's ndcg@5: 0.397601
[1413]	valid_0's ndcg@1: 0.2115	valid_0's ndcg@2: 0.29356	valid_0's ndcg@3: 0.340901	valid_0's ndcg@4: 0.373222	valid_0's ndcg@5: 0.397621
[1414]	valid_0's ndcg@1: 0.211452	valid_0's ndcg@2: 0.293624	valid_0's ndcg@3: 0.340836	valid_0's ndcg@4: 0.373212	valid_0's ndcg@5: 0.39761
[1415]	valid_0's ndcg@1: 0.211452	valid_0's ndcg@2: 0.293561	valid_0's ndcg@3: 0.340774	valid_0's ndcg@4: 0.373192	valid_0's ndcg@5: 0.39759
[1416]	valid_0's ndcg@1: 0.211452	valid_0's ndcg@2: 0.293561	valid_0's ndcg@3: 0.340749	valid_0's ndcg@4: 0.373146	valid_0's ndcg@5: 0.397564
[1417]	vali

[1468]	valid_0's ndcg@1: 0.212149	valid_0's ndcg@2: 0.293265	valid_0's ndcg@3: 0.340746	valid_0's ndcg@4: 0.373309	valid_0's ndcg@5: 0.397767
[1469]	valid_0's ndcg@1: 0.21205	valid_0's ndcg@2: 0.293198	valid_0's ndcg@3: 0.340704	valid_0's ndcg@4: 0.373244	valid_0's ndcg@5: 0.397741
[1470]	valid_0's ndcg@1: 0.21205	valid_0's ndcg@2: 0.293104	valid_0's ndcg@3: 0.340709	valid_0's ndcg@4: 0.373185	valid_0's ndcg@5: 0.397739
[1471]	valid_0's ndcg@1: 0.21205	valid_0's ndcg@2: 0.293104	valid_0's ndcg@3: 0.340684	valid_0's ndcg@4: 0.373203	valid_0's ndcg@5: 0.397788
[1472]	valid_0's ndcg@1: 0.212001	valid_0's ndcg@2: 0.292991	valid_0's ndcg@3: 0.340612	valid_0's ndcg@4: 0.37311	valid_0's ndcg@5: 0.397606
[1473]	valid_0's ndcg@1: 0.211951	valid_0's ndcg@2: 0.292942	valid_0's ndcg@3: 0.340538	valid_0's ndcg@4: 0.373079	valid_0's ndcg@5: 0.397593
[1474]	valid_0's ndcg@1: 0.211802	valid_0's ndcg@2: 0.292887	valid_0's ndcg@3: 0.340532	valid_0's ndcg@4: 0.373073	valid_0's ndcg@5: 0.397512
[1475]	val

LGBMRanker(deterministic=True, extra_trees=True, metric='ndcg',
           num_boost_round=3000, objective='lambdarank', seed=12345)

In [11]:
gbm.get_params()

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.1,
 'max_depth': -1,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 100,
 'n_jobs': -1,
 'num_leaves': 31,
 'objective': 'lambdarank',
 'random_state': None,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'silent': True,
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0,
 'metric': 'ndcg',
 'num_boost_round': 3000,
 'extra_trees': True,
 'seed': 12345,
 'deterministic': True}

In [12]:
gc.collect()
with open('model/model', 'wb') as f:
    pickle.dump(gbm, f)

In [13]:
with open('model/model', 'rb') as f:
    model = pickle.load(f)

In [14]:
del x
del y
del x_val
del y_val
gc.collect()

20

In [15]:
test = pd.read_csv("data/dummy/test.csv", index_col=False)
test.drop(['date_time', 'random_bool'], axis=1, inplace=True)
srch_ids = test['srch_id']
test.drop('srch_id', axis=1, inplace=True)

In [16]:
test

,site_id,visitor_location_country_id,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,price_usd,promotion_flag,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,orig_destination_distance,agg_comp_rate,agg_comp_perc,agg_comp_inv,total,price_usd_prop_id_mean,price_usd_prop_id_std,price_usd_prop_id_median,prop_starrating_prop_id_mean,prop_review_score_prop_id_mean,prop_location_score1_prop_id_mean,prop_location_score1_prop_id_median,prop_location_score2_prop_id_mean,prop_location_score2_prop_id_std,prop_location_score2_prop_id_median,price_usd_srch_id_mean,price_usd_srch_id_std,price_usd_srch_id_median,prop_starrating_srch_id_std,prop_review_score_srch_id_std,prop_location_score1_srch_id_std,prop_location_score2_srch_id_mean,prop_location_score2_srch_id_std,prop_location_score2_srch_id_median,price_usd_srch_destination_id_median,prop_location_score2_srch_destination_id_mean,prop_location_score2_srch_destination_id_std,prop_location_score2_srch_destination_id_median
0,24,216,219,3180,3,4.5,1,2.94,0.0691,5.03,119.00,0,19222,1,10,2,0,1,0,-1.0,0.00,0.000000,0.0,2,125.520757,11.741938,125.000,3,4.5,2.94,2.94,0.039134,0.028124,0.0251,93.796552,36.937113,84.15,0.712313,0.940011,0.300962,0.125383,0.076598,0.0908,82.00,0.113938,0.079721,0.0895
1,24,216,219,5543,3,4.5,1,2.64,0.0843,4.93,118.00,0,19222,1,10,2,0,1,0,-1.0,0.00,0.000000,0.0,2,117.532579,20.524552,116.000,3,4.5,2.64,2.64,0.035593,0.033294,0.0106,93.796552,36.937113,84.15,0.712313,0.940011,0.300962,0.125383,0.076598,0.0908,82.00,0.113938,0.079721,0.0895
2,24,216,219,14142,2,3.5,1,2.71,0.0556,4.16,49.00,0,19222,1,10,2,0,1,0,-1.0,0.50,10.000000,0.0,2,51.886600,4.478971,49.000,2,3.5,2.71,2.71,0.034479,0.021327,0.0248,93.796552,36.937113,84.15,0.712313,0.940011,0.300962,0.125383,0.076598,0.0908,82.00,0.113938,0.079721,0.0895
3,24,216,219,22393,3,4.5,1,2.40,0.0561,5.03,143.00,0,19222,1,10,2,0,1,0,-1.0,0.00,0.000000,0.0,2,133.987619,28.061057,139.000,3,4.5,2.40,2.40,0.034571,0.022254,0.0202,93.796552,36.937113,84.15,0.712313,0.940011,0.300962,0.125383,0.076598,0.0908,82.00,0.113938,0.079721,0.0895
4,24,216,219,24194,3,4.5,1,2.94,0.2090,4.72,79.00,0,19222,1,10,2,0,1,0,-1.0,0.00,0.000000,0.0,2,86.308224,14.842057,85.000,3,4.5,2.94,2.94,0.110409,0.085862,0.0712,93.796552,36.937113,84.15,0.712313,0.940011,0.300962,0.125383,0.076598,0.0908,82.00,0.113938,0.079721,0.0895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4959178,24,216,117,32019,4,3.5,0,2.48,0.0551,4.53,66.07,0,19246,2,7,1,0,1,0,-1.0,0.50,49.500000,0.0,1,62.260769,1.441622,62.000,4,3.5,2.48,2.48,0.031285,0.041353,0.0005,76.424286,15.748111,70.05,0.377964,2.110067,0.369498,0.109143,0.116623,0.0551,70.05,0.109143,0.116623,0.0551
4959179,24,216,117,33959,4,3.0,1,2.20,0.3344,4.39,67.10,0,19246,2,7,1,0,1,0,-1.0,0.25,19.000000,0.0,1,63.847692,13.486010,59.720,4,3.0,2.20,2.20,0.026315,0.092568,0.0007,76.424286,15.748111,70.05,0.377964,2.110067,0.369498,0.109143,0.116623,0.0551,70.05,0.109143,0.116623,0.0551
4959180,24,216,117,35240,4,0.0,0,1.79,0.0000,4.64,73.91,0,19246,2,7,1,0,1,0,-1.0,0.25,24.666667,0.0,1,62.955000,15.492710,62.955,4,0.0,1.79,1.79,0.000100,0.000141,0.0001,76.424286,15.748111,70.05,0.377964,2.110067,0.369498,0.109143,0.116623,0.0551,70.05,0.109143,0.116623,0.0551
4959181,24,216,117,94437,4,0.0,0,2.94,0.0928,4.64,66.07,0,19246,2,7,1,0,1,0,-1.0,0.00,27.500000,0.0,1,64.810000,6.670887,68.110,4,0.0,2.94,2.94,0.014029,0.034735,0.0009,76.424286,15.748111,70.05,0.377964,2.110067,0.369498,0.109143,0.116623,0.0551,70.05,0.109143,0.116623,0.0551


In [ ]:
predictions = model.predict(test)

In [ ]:
len(predictions)

In [ ]:
#submission = test[['srch_id', 'prop_id']]
submission = pd.concat([srch_ids, test['prop_id']], axis=1)
submission['pred'] = predictions

In [ ]:
submission.head(n=100)

In [ ]:
del test
gc.collect()

In [ ]:
submission = submission.sort_values(['srch_id', 'pred'], ascending=[True, False])

In [ ]:
submission.head(n=20)

In [ ]:
submission[['srch_id', 'prop_id']].to_csv('submission.csv', index=False)